ML Model for Soccer Players

Attempting to model the price of players based on stats

In [6]:
import pandas as pd
import numpy as np
from scipy import *
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import plotly.graph_objects as go

In [7]:
# Import data
games_df = pd.read_csv('games.csv')
players_df = pd.read_csv('players.csv')
valuations_df = pd.read_csv('player_valuations.csv')
appearances_df = pd.read_csv('appearances.csv')

In [8]:
players_df = players_df[(players_df['height_in_cm']>100)]
players_df = players_df[['player_id', 'height_in_cm', 'position', 'foot', 'highest_market_value_in_eur', 
                         'country_of_citizenship', 'country_of_birth', 'date_of_birth', 'sub_position', 
                         'last_season', 'current_club_id']]

game_stats = appearances_df[['player_id', 'yellow_cards', 'red_cards', 'goals', 'assists']]
game_stats = game_stats.groupby(game_stats['player_id']).aggregate('sum')

players_df = players_df.set_index(['player_id'])

all_stats = pd.merge(game_stats, players_df, on='player_id')

all_stats = pd.get_dummies(all_stats)

print(all_stats)

all_stats = all_stats.dropna()

           yellow_cards  red_cards  goals  assists  height_in_cm  \
player_id                                                          
10                   19          0     48       24         184.0   
26                    4          2      0        0         190.0   
80                    0          0      0        0         194.0   
132                  13          0      9        4         179.0   
215                   3          0     26        8         193.0   
...                 ...        ...    ...      ...           ...   
1143811               1          0      0        0         181.0   
1143823               0          0      0        0         173.0   
1159449               0          0      0        0         180.0   
1163778               0          0      0        0         193.0   
1166093               1          0      1        0         182.0   

           highest_market_value_in_eur  last_season  current_club_id  \
player_id                                  

In [9]:
# Partition Data
X = all_stats.loc[:, all_stats.columns != 'highest_market_value_in_eur']
y = all_stats['highest_market_value_in_eur']

print(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, train_size=0.8) 

           yellow_cards  red_cards  goals  assists  height_in_cm  last_season  \
player_id                                                                       
10                   19          0     48       24         184.0         2015   
26                    4          2      0        0         190.0         2017   
80                    0          0      0        0         194.0         2017   
132                  13          0      9        4         179.0         2015   
215                   3          0     26        8         193.0         2015   
...                 ...        ...    ...      ...           ...          ...   
1134676               0          0      0        0         174.0         2023   
1137664               0          0      0        0         180.0         2023   
1141628               1          0      0        1         181.0         2023   
1159449               0          0      0        0         180.0         2023   
1166093               1     

In [10]:
# KNN Model
param_grid_knn = {
    'n_neighbors': [15]                                 
}
knn = GridSearchCV(estimator=KNeighborsClassifier(), 
                   param_grid=param_grid_knn, 
                   verbose=1, cv=10, n_jobs=1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(knn.score(X_test, y_test))

Fitting 10 folds for each of 1 candidates, totalling 10 fits


c:\Users\lenon\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


0.05987233105877174
